In [1]:
import logging

from libs.configuration import configure
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.sql.avro import functions as AF

from shared.spark_config import create_spark_config

__MODULE = "M2_Processors.airline.raw"
logger = logging.getLogger("notebook")
env = configure()
conf = create_spark_config().setAppName(__MODULE)

{'DATASTORAGE_AZURE_ACCESSKEY': 'dYj3PCGRyYHYR9pRo2xPa9BQ7t/sUonACJ29wcPdpa+1IC70rMbZmpfpHJJWJtk7OeASL+eQDIWq+AStGg3CCA==',
 'DATASTORAGE_AZURE_ACCOUNTNAME': 'icebergpe6t5cug7x9f45p8',
 'KAFKA_BOOTSTRAP_SERVERS': 'localhost:9092',
 'KAFKA_SCHEMA_REGISTRY_URL': 'http://localhost:8081',
 'KAFKA_SESSION_TIMEOUT_MS': '45000',
 'KAFKA_TOPIC_RAW_AIRCRAFT': 'raw.aircraft',
 'KAFKA_TOPIC_RAW_AIRLINE': 'raw.airline',
 'KAFKA_TOPIC_RAW_AIRPORT': 'raw.airport',
 'KAFKA_TOPIC_RAW_FLIGHT': 'raw.flight',
 'KAFKA_TOPIC_T1_AIRCRAFT': 't1.aircraft',
 'KAFKA_TOPIC_T1_AIRLINE': 't1.airline',
 'KAFKA_TOPIC_T1_AIRPORT': 't1.airport',
 'KAFKA_TOPIC_T1_FLIGHT': 't1.flight',
 'KAFKA_TOPIC_T2_AIRCRAFT': 't2.aircraft',
 'KAFKA_TOPIC_T2_AIRLINE': 't2.airline',
 'KAFKA_TOPIC_T2_AIRPORT': 't2.airport',
 'KAFKA_TOPIC_T2_FLIGHT': 't2.flight',
 'POSTGRE_JDBC_URL': 'jdbc:postgresql://localhost:5432/iceberg-catalog',
 'POSTGRE_PASSWORD': 'grebeci',
 'POSTGRE_USER': 'iceberg',
 'SPARK_MASTER_URL': 'local[1]',
 'file': '

In [2]:
spark = SparkSession.builder.config(conf=conf).getOrCreate()

your 131072x1 screen size is bogus. expect trouble
24/01/05 21:45:11 WARN Utils: Your hostname, rbtwmachine-pc resolves to a loopback address: 127.0.1.1; using 192.168.0.10 instead (on interface eth0)
24/01/05 21:45:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/neot/Work/de-rbearthgaze/M2_Processors/.venv/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/neot/.ivy2/cache
The jars for the packages stored in: /home/neot/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.spark#spark-avro_2.12 added as a dependency
org.apache.hadoop#hadoop-common added as a dependency
org.apache.hadoop#hadoop-azure added as a dependency
com.azure#azure-storage-blob added as a dependency
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-3389bed0-c2e2-4935-b8c5-3ddfa76c616a;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.4.3 in central
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.0 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.3 in centr

In [6]:
spark.sql("SHOW CATALOGS").show()

+-------------+
|      catalog|
+-------------+
|          dev|
|spark_catalog|
+-------------+



In [7]:
spark.sql("""
CREATE TABLE IF NOT EXISTS dev.raw.airlines (
    ingested_at timestamp,
    id int,
    name string,
    alias string,
    iata string,
    icao string,
    callsign string,
    country string,
    active string
)
USING iceberg
""").show()

++
||
++
++



In [10]:
spark.sql("""
CREATE TABLE IF NOT EXISTS dev.tier1.airlines (
    ingested_at timestamp,
    icao string,
    iata string,
    name string,
    alias string,
    callsign string,
    country string,
    active string
)
USING iceberg
""").show()

++
||
++
++



In [15]:
spark.sql("""
CREATE TABLE IF NOT EXISTS dev.tier2.airlines (
    ingested_at timestamp,
    icao string,
    iata string,
    name string,
    alias string,
    callsign string,
    country string,
    active string
)
USING iceberg
PARTITIONED BY (country)
""").show()

++
||
++
++



In [16]:
spark.sql("ALTER TABLE dev.tier2.airlines WRITE ORDERED BY icao").show()

DataFrame[]